## Librerias y parametros iniciales

In [19]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import folium
import random
import os
import json
import warnings

warnings.filterwarnings("ignore")

archivo_entrada = 'tsp_con_coordenadas.xlsx'
path_de_redes = 'redes'
path_de_resultados = 'resultados'

os.makedirs(path_de_redes, exist_ok=True)
os.makedirs(path_de_resultados, exist_ok=True)

df_tsp = pd.read_excel(archivo_entrada)

## Generar redes

Se generan las siguientes redes:

- 10 redes con 40 nodos
- 10 redes con 100 nodos
- 10 redes con 150 nodos
- 10 redes con 200 nodos


In [20]:
generar = False  # Cambiar a True para generar las redes si es necesario

configuraciones = {
    40: 10,
    100: 10,
    150: 10,
    200: 10
}

def generar_redes():
    numero_red = 1

    for nodos, cantidad in configuraciones.items():
        for _ in range(cantidad):
            # Crear un dataframe vacío con la misma estructura que el original
            columnas = df_tsp.columns
            red_df = pd.DataFrame(columns=columnas)

            # Lista de índices seleccionados para evitar duplicados
            nodos_seleccionados = set()

            # Seleccionar nodos aleatorios
            while len(red_df) < nodos:
                nodo = df_tsp.sample(n=1, random_state=random.randint(1, 10000))
                if nodo.index[0] not in nodos_seleccionados:
                    nodos_seleccionados.add(nodo.index[0])
                    red_df = pd.concat([red_df, nodo], ignore_index=True)

            # Guardar el DataFrame en un archivo Excel
            nombre_archivo = f"redes/{numero_red}_{nodos}.xlsx"
            red_df.to_excel(nombre_archivo, index=False)

            # Incrementar el número de red
            numero_red += 1

if generar == True:
    generar_redes()

## Funciones auxiliares

In [21]:
def haversine_distance(coord1, coord2):
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = sin(dlat/2)**2 + cos(lat1_rad)*cos(lat2_rad)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def crear_matriz_distancias(df_red):
    coordenadas = df_red[['latitud', 'longitud']].values
    n = len(coordenadas)
    matriz_distancias = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            distancia = haversine_distance(coordenadas[i], coordenadas[j])
            matriz_distancias[i, j] = distancia
            matriz_distancias[j, i] = distancia

    return matriz_distancias

## Algoritmo genético


In [22]:
def inicializar_poblacion(tamano_poblacion, indices_ciudades):
    poblacion = []
    for _ in range(tamano_poblacion):
        individuo = indices_ciudades.copy()
        random.shuffle(individuo)
        poblacion.append(individuo)
    return poblacion

def calcular_aptitud(individuo, matriz_distancias):
    distancia_total = 0
    for i in range(len(individuo)):
        ciudad_origen = individuo[i]
        ciudad_destino = individuo[(i + 1) % len(individuo)]
        distancia_total += matriz_distancias[ciudad_origen][ciudad_destino]
    return distancia_total

def seleccionar_padres(poblacion, aptitudes, num_padres):
    padres_indices = np.argsort(aptitudes)[:num_padres]
    padres = [poblacion[i] for i in padres_indices]
    return padres

def cruzar_padres(padre1, padre2):
    tamano = len(padre1)
    inicio, fin = sorted(random.sample(range(tamano), 2))
    hijo = [None]*tamano
    hijo[inicio:fin+1] = padre1[inicio:fin+1]

    pointer = 0
    for i in range(tamano):
        if hijo[i] is None:
            while padre2[pointer] in hijo:
                pointer += 1
            hijo[i] = padre2[pointer]
            pointer += 1
    return hijo

def mutar_individuo(individuo, tasa_mutacion):
    for i in range(len(individuo)):
        if random.random() < tasa_mutacion:
            j = random.randint(0, len(individuo)-1)
            individuo[i], individuo[j] = individuo[j], individuo[i]
    return individuo

def algoritmo_genetico_tsp(matriz_distancias, tamano_poblacion=100, num_generaciones=100, tasa_mutacion=0.01):
    indices_ciudades = list(range(len(matriz_distancias)))
    poblacion = inicializar_poblacion(tamano_poblacion, indices_ciudades)
    mejor_distancia = float('inf')
    mejor_ruta = None

    for generacion in range(num_generaciones):
        aptitudes = [calcular_aptitud(individuo, matriz_distancias) for individuo in poblacion]
        distancia_actual = min(aptitudes)
        if distancia_actual < mejor_distancia:
            mejor_distancia = distancia_actual
            mejor_ruta = poblacion[np.argmin(aptitudes)]

        num_padres = tamano_poblacion // 2
        padres = seleccionar_padres(poblacion, aptitudes, num_padres)

        siguiente_generacion = []
        while len(siguiente_generacion) < tamano_poblacion:
            padre1, padre2 = random.sample(padres, 2)
            hijo = cruzar_padres(padre1, padre2)
            hijo = mutar_individuo(hijo, tasa_mutacion)
            siguiente_generacion.append(hijo)

        poblacion = siguiente_generacion

        if (generacion + 1) % 50 == 0:
            print(f"Generación {generacion + 1}: Mejor distancia = {mejor_distancia:.2f} km")

    return mejor_ruta, mejor_distancia

## Algoritmo colonia de hormigas (ACO)

In [23]:
def calcular_heuristica(costos):
    num_nodos = costos.shape[0]
    heuristica = np.zeros_like(costos)
    for i in range(num_nodos):
        for j in range(num_nodos):
            if costos[i, j] > 0 and not np.isinf(costos[i, j]):
                heuristica[i, j] = 1 / costos[i, j]
            else:
                heuristica[i, j] = 0
    return heuristica

def calcular_probabilidades(feromonas, heuristica, nodo_actual, visitados, alpha, beta):
    num_nodos = len(feromonas)
    probabilidades = np.zeros(num_nodos)
    for j in range(num_nodos):
        if j not in visitados:
            probabilidades[j] = (feromonas[nodo_actual, j] ** alpha) * (heuristica[nodo_actual, j] ** beta)
    suma_probabilidades = np.sum(probabilidades)
    if suma_probabilidades == 0:
        return probabilidades
    return probabilidades / suma_probabilidades

def construir_camino(feromonas, heuristica, costos, alpha, beta):
    num_nodos = costos.shape[0]
    nodo_actual = random.randint(0, num_nodos - 1)
    camino = [nodo_actual]
    costo_total = 0
    visitados = set(camino)

    while len(camino) < num_nodos:
        probabilidades = calcular_probabilidades(feromonas, heuristica, nodo_actual, visitados, alpha, beta)
        if np.sum(probabilidades) == 0:
            nodos_no_visitados = list(set(range(num_nodos)) - visitados)
            siguiente_nodo = random.choice(nodos_no_visitados)
        else:
            siguiente_nodo = random.choices(range(num_nodos), weights=probabilidades, k=1)[0]
        costo_total += costos[nodo_actual, siguiente_nodo]
        camino.append(siguiente_nodo)
        visitados.add(siguiente_nodo)
        nodo_actual = siguiente_nodo

    costo_total += costos[camino[-1], camino[0]]
    camino.append(camino[0])

    return camino, costo_total

def actualizar_feromonas(feromonas, soluciones, rho):
    feromonas *= (1 - rho)
    for camino, costo in soluciones:
        for i in range(len(camino) - 1):
            feromonas[camino[i], camino[i + 1]] += 1 / costo

def ACO(costos, num_hormigas=500, num_iteraciones=100, alpha=1.0, beta=2.0, rho=0.5):
    num_nodos = costos.shape[0]
    mejor_camino = None
    mejor_costo = float('inf')

    feromonas = np.ones((num_nodos, num_nodos)) * 0.1
    heuristica = calcular_heuristica(costos)

    for iteracion in range(num_iteraciones):
        soluciones = []
        for _ in range(num_hormigas):
            camino, costo = construir_camino(feromonas, heuristica, costos, alpha, beta)
            soluciones.append((camino, costo))
            if costo < mejor_costo:
                mejor_camino = camino
                mejor_costo = costo

        actualizar_feromonas(feromonas, soluciones, rho)

        if (iteracion + 1) % 10 == 0:
            print(f"Iteración {iteracion+1}: Mejor costo hasta ahora = {mejor_costo:.2f} km")

    return mejor_camino, mejor_costo

## Procesar las redes

In [24]:
def procesar_redes(archivo):
    df_red = pd.read_excel(archivo)

    matriz_distancias = crear_matriz_distancias(df_red)

    mejor_ruta_genetico, mejor_distancia_genetico = algoritmo_genetico_tsp(matriz_distancias, tamano_poblacion=100, num_generaciones=500, tasa_mutacion=0.01)
    mejor_ruta_aco, mejor_distancia_aco = ACO(matriz_distancias, num_hormigas=500, num_iteraciones=100, alpha=1.0, beta=2.0, rho=0.5)

    return mejor_ruta_genetico, mejor_distancia_genetico, mejor_ruta_aco, mejor_distancia_aco

def crear_visualizacion(df_red, mejor_ruta, nombre_archivo):
    coordenadas = df_red[['latitud', 'longitud']].values
    ruta_coordenadas = coordenadas[mejor_ruta]
    ruta_coordenadas = np.vstack([ruta_coordenadas, ruta_coordenadas[0]])

    centro_lat = np.mean(coordenadas[:, 0])
    centro_lon = np.mean(coordenadas[:, 1])
    mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

    for idx, coord in enumerate(ruta_coordenadas[:-1]):
        folium.Marker(location=coord, popup=f"Parada {idx + 1}").add_to(mapa)

    folium.PolyLine(ruta_coordenadas, color="blue", weight=2.5, opacity=1).add_to(mapa)

    return mapa.save(nombre_archivo)

def generar_archivos_inclusion(archivo):
        df_red = pd.read_excel(archivo)
        matriz_distancias = crear_matriz_distancias(df_red)
        n = len(df_red)
        nodos = [f"n{idx}" for idx in range(n)]  # Nombres de nodos

        # Crear el archivo de inclusión
        nombre_archivo_inc = f"{path_de_resultados}/{archivo.split('/')[-1].split('.')[0]}/gams_{archivo.split('/')[-1].split('.')[0]}.inc"
        with open(nombre_archivo_inc, 'w') as f:
            # Escribir encabezado (índices de columnas)
            f.write("    ")  # Espacio para el índice de filas
            for j in range(n):
                f.write(f"{j:<8}")  # Alinear columnas con espacio fijo
            f.write("\n")

            # Escribir filas de la matriz
            for i in range(n):
                f.write(f"{i:<4}")  # Índice de la fila
                for j in range(n):
                    f.write(f"{matriz_distancias[i][j]:<8.1f}")  # Valor de la matriz con 1 decimal
                f.write("\n")

            # Escribir parámetros adicionales
            # f.write("\nPARAMETER direccion(i) /\n")
            # for i in range(n):
            #     numero = df_red.loc[i, 'Numero']
            #     f.write(f"  {nodos[i]} {numero}\n")
            # f.write("/;\n")

        print(f"Archivo de inclusión generado: {nombre_archivo_inc}")

def procesar_archivo(archivo):
    print(f"Procesando archivo: {archivo}")
    df_red = pd.read_excel(archivo)

    path_de_resultado_base = f"{path_de_resultados}/{archivo.split('/')[-1].split('.')[0]}"
    os.makedirs(path_de_resultado_base, exist_ok=True)

    path_de_resultado_json = f"{path_de_resultado_base}/{archivo.split('/')[-1].split('.')[0]}.json"
    path_de_visualizacion_genetico = f"{path_de_resultado_base}/{archivo.split('/')[-1].split('.')[0]}_genetico.html"
    path_de_visualizacion_aco = f"{path_de_resultado_base}/{archivo.split('/')[-1].split('.')[0]}_aco.html"

    mejor_ruta_genetico, mejor_distancia_genetico, mejor_ruta_aco, mejor_distancia_aco = procesar_redes(archivo)

    # Generar resultados en un archivo json
    with open(path_de_resultado_json, "w") as f:
        json.dump({
            "genetico": {
                "ruta": mejor_ruta_genetico,
                "distancia": mejor_distancia_genetico
            },
            "aco": {
                "ruta": mejor_ruta_aco,
                "distancia": mejor_distancia_aco
            }
        }, f)

    # Crear visualización de la mejor ruta encontrada por el algoritmo genético
    crear_visualizacion(df_red, mejor_ruta_genetico, path_de_visualizacion_genetico)
    crear_visualizacion(df_red, mejor_ruta_aco, path_de_visualizacion_aco)
    print(f"Resultados guardados en: {path_de_resultado_base}\n\n")

# Procesar cada archivo de red
paths = [f"{path_de_redes}/{archivo}" for archivo in os.listdir(path_de_redes)]
paths_sorted = sorted(paths, key=lambda x: int(x.split('/')[1].split('_')[0]))

for archivo in paths_sorted:
    # procesar_archivo(archivo)
    generar_archivos_inclusion(archivo)

Archivo de inclusión generado: resultados/1_40/gams_1_40.inc
Archivo de inclusión generado: resultados/2_40/gams_2_40.inc
Archivo de inclusión generado: resultados/3_40/gams_3_40.inc
Archivo de inclusión generado: resultados/4_40/gams_4_40.inc
Archivo de inclusión generado: resultados/5_40/gams_5_40.inc
Archivo de inclusión generado: resultados/6_40/gams_6_40.inc
Archivo de inclusión generado: resultados/7_40/gams_7_40.inc
Archivo de inclusión generado: resultados/8_40/gams_8_40.inc
Archivo de inclusión generado: resultados/9_40/gams_9_40.inc
Archivo de inclusión generado: resultados/10_40/gams_10_40.inc
Archivo de inclusión generado: resultados/11_100/gams_11_100.inc
Archivo de inclusión generado: resultados/12_100/gams_12_100.inc
Archivo de inclusión generado: resultados/13_100/gams_13_100.inc
Archivo de inclusión generado: resultados/14_100/gams_14_100.inc
Archivo de inclusión generado: resultados/15_100/gams_15_100.inc
Archivo de inclusión generado: resultados/16_100/gams_16_100.in